<a href="https://colab.research.google.com/github/aaalexlit/wandb_notebooks/blob/main/W%26B_03_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb -qq
!wget -nc https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py
!wget -nc https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py

In [2]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU
     

In [3]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

In [4]:
set_seed(train_config.seed, reproducible=True)

In [5]:
run = wandb.init(project=params.WANDB_PROJECT,
                 entity=params.ENTITY,
                 job_type='training',
                 config=train_config)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.24MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:1:30.2


## Remove test data

In [7]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [8]:
# Get mask filename from the current image filename
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [9]:
# Paths to images and masks
df['image_fname'] = [processed_dataset_dir/f'images/{f}.jpg' for f in df.File_Name.values]
df['label_fname'] = [label_func(f) for f in df.image_fname.values]

In [10]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [11]:
config = wandb.config

In [12]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

In [13]:
metrics = [MIOU(),
           BackgroundIOU(),
           RoadIOU(),
           TrafficLightIOU(),
           TrafficSignIOU(), 
           PersonIOU(),
           VehicleIOU(),
           BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 152MB/s]


In [14]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

In [15]:
learn.fit_one_cycle(config.epochs, 
                    config.lr,
                    cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.533055,0.443764,0.268942,0.822098,0.514706,0.000000,0.000000,0.000000,0.545787,0.000000,00:45
1,0.589067,0.780535,0.096392,0.674747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,00:42
2,0.440438,0.361635,0.313703,0.846465,0.736176,0.000000,0.000000,0.000000,0.613277,0.000000,00:41
3,0.340346,0.275802,0.338493,0.894783,0.802930,0.000000,0.000000,0.000000,0.671739,0.000000,00:41
4,0.288074,0.269498,0.341863,0.896194,0.800876,0.000000,0.000000,0.000000,0.695973,0.000000,00:41
5,0.249714,0.259750,0.346226,0.901510,0.818540,0.000000,0.000000,0.000000,0.703535,0.000000,00:41
6,0.238051,0.254287,0.351018,0.908059,0.820139,0.000000,0.000000,0.000000,0.728926,0.000000,00:41
7,0.212776,0.234792,0.358091,0.914548,0.833941,0.016417,0.000000,0.000000,0.741735,0.000000,00:41
8,0.197280,0.235630,0.363951,0.916177,0.834917,0.045361,0.000000,0.000000,0.751200,0.000000,00:41
9,0.186777,0.234537,0.363615,0.917177,0.836948,0.042482,0.000000,0.000000,0.748696,0.000000,00:41


Better model found at epoch 0 with miou value: 0.26894151664824123.
Better model found at epoch 2 with miou value: 0.31370260105093467.
Better model found at epoch 3 with miou value: 0.3384930815707454.
Better model found at epoch 4 with miou value: 0.34186325527449885.
Better model found at epoch 5 with miou value: 0.34622640906947727.
Better model found at epoch 6 with miou value: 0.35101778080140883.
Better model found at epoch 7 with miou value: 0.35809145637927126.
Better model found at epoch 8 with miou value: 0.36395083131367595.


In [16]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({'pred_table': table})

We are reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we will validate the model again and save the final loss and metrics to wandb.summary.

In [17]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}

for k, v in final_results.items():
  wandb.summary[k] = v

In [18]:
wandb.finish()

background_iou,▅▁▆▇▇█████
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
lr_2,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
miou,▆▁▇▇▇█████
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
